In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

def turning_point_test(series):
    """
    Performs the Turning Point test for randomness.
    Returns a dictionary with test statistics and decision.
    """
    x = np.array(series)
    x = x[~np.isnan(x)]  # Remove NaN values
    N = len(x)

    if N < 3:
        return {
            "N": N,
            "SN (Turning Points)": np.nan,
            "Mean µ": np.nan,
            "Variance σ²": np.nan,
            "Z (U value)": np.nan,
            "Decision": "Not Enough Data"
        }

    # Step 1: Identify turning points
    SN = 0
    for i in range(1, N-1):
        if (x[i-1] < x[i] > x[i+1]) or (x[i-1] > x[i] < x[i+1]):
            SN += 1

    # Step 2: Expected mean
    mu = (2/3) * (N - 2)

    # Step 3: Variance
    sigma2 = (16*N - 29) / 90
    sigma = np.sqrt(sigma2)

    # Step 4: Z-statistic
    if sigma == 0:
        U = 0.0
    else:
        U = (SN - mu) / sigma

    # Step 5: Randomness decision (based on 95% confidence level, Z=1.96)
    if abs(U) < 1.96:
        result = "✅ Random"
    else:
        result = "❌ Not Random"

    return {
        "N": N,
        "SN (Turning Points)": SN,
        "Mean µ": round(mu, 2),
        "Variance σ²": round(sigma2, 2),
        "Z (U value)": round(U, 2),
        "Decision": result
    }

# ---- EDIT FILE PATHS ----
file_path = Path(r"D:\climate change\monthly_averages.xlsx")
output_path = Path(r"D:\climate change\TurningPoint_Randomness_AllSheets.csv")

# Load all sheets in Excel file
try:
    xls = pd.ExcelFile(file_path)
    print(f"Successfully loaded {file_path}")
except FileNotFoundError:
    print(f"❌ ERROR: File not found at {file_path}")
    exit()
except Exception as e:
    print(f"❌ ERROR: Could not load Excel file. {e}")
    exit()

print("Running Turning Point Test for Randomness")
print("="*80)
print("(Null Hypothesis: The data series is random)")
print("|Z| < 1.96: Cannot reject null hypothesis (✅ Random)")
print("|Z| >= 1.96: Reject null hypothesis (❌ Not Random)")
print("="*80)

results = []

for sheet in xls.sheet_names:
    print(f"\n📄 Sheet: {sheet}")
    try:
        df = pd.read_excel(file_path, sheet_name=sheet)
    except Exception as e:
        print(f"  ...Could not read sheet '{sheet}'. Error: {e}")
        continue

    # Select only numeric columns
    num_df = df.select_dtypes(include=[np.number])
    
    # Drop Lat/Lon if they exist, as they are not time series
    num_df = num_df.drop(columns=["Lon", "Lat"], errors="ignore")

    if num_df.empty:
        print("  ...No numeric data columns found in this sheet.")
        continue

    for col in num_df.columns:
        res = turning_point_test(num_df[col])
        res["Sheet"] = sheet
        res["Station"] = col
        results.append(res)
    
    print(f"  ...processed {len(num_df.columns)} stations.")

# Save all results to one CSV
try:
    df_results = pd.DataFrame(results)
    
    # Reorder columns to be more readable
    cols = ["Sheet", "Station", "Decision", "Z (U value)", "SN (Turning Points)", "Mean µ", "Variance σ²", "N"]
    # Ensure all columns are present, even if results were empty
    df_results = df_results.reindex(columns=cols, fill_value=np.nan) 
    
    df_results.to_csv(output_path, index=False)
    print(f"\n✅ All results saved to:\n{output_path}")
except Exception as e:
    print(f"\n❌ ERROR: Could not save CSV file. {e}")

print("\n...Done.")


Successfully loaded D:\climate change\monthly_averages.xlsx
Running Turning Point Test for Randomness
(Null Hypothesis: The data series is random)
|Z| < 1.96: Cannot reject null hypothesis (✅ Random)
|Z| >= 1.96: Reject null hypothesis (❌ Not Random)

📄 Sheet: Rainfall_monthly
  ...processed 23 stations.

📄 Sheet: Tmax_monthly
  ...processed 23 stations.

📄 Sheet: Tmin_monthly
  ...processed 23 stations.

✅ All results saved to:
D:\climate change\TurningPoint_Randomness_AllSheets.csv

...Done.
